In [1]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.SeqUtils import nt_search
from Bio.SeqIO import write
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import pairwise_distances
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def convert_to_three_letter(aa):
    """Convert single-letter amino acid code to three-letter code."""
    aa_dict = {
        "A" : "ALA", "R" : "ARG", "N" : "ASN", "D" : "ASP", "C" : "CYS", 
        "Q" : "GLN", "E" : "GLU", "G" : "GLY", "H" : "HIS", "I" : "ILE", 
        "L" : "LEU", "K" : "LYS", "M" : "MET", "F" : "PHE", "P" : "PRO", 
        "S" : "SER", "T" : "THR", "W" : "TRP", "Y" : "TYR", "V" : "VAL"
    }
    return aa_dict.get(aa, "Unknown")

def compare_sequences(test_seq, ref_seq, start_pos):
    """
    Compare two sequences and identify mutations, adjusting positions with an offset.
    
    Parameters:
    - test_seq: str, the sequence to be tested
    - ref_seq: str, the reference sequence
    - start_pos: int, start_pos offset for mutation numbering
    
    Returns:
    - str: A string of mutations in the format "A:<position> <reference_3letter> -> <test_3letter>"
    """
    # Ensure the sequences are the same length
    if len(test_seq) != len(ref_seq):
        raise ValueError("Sequences must be of the same length")
    
    # Initialize a list to store positions of differences
    mutations = []
    
    # Loop through each position and compare residues
    for i in range(len(ref_seq)):
        # Skip positions where either sequence has 'X'
        if ref_seq[i] == "X" or test_seq[i] == "X":
            continue
        if ref_seq[i] != test_seq[i]:
            ref_aa_3letter = convert_to_three_letter(ref_seq[i])
            test_aa_3letter = convert_to_three_letter(test_seq[i])
            mutations.append(f"A:{i + start_pos}->{test_aa_3letter}")
    
    # Return the mutations as a single string
    return " ".join(mutations)


In [3]:
# Read sequences from the FASTA file
def read_sequences(file_path):
    """Read sequences from a FASTA file."""
    sequences = list(SeqIO.parse(file_path, "fasta"))
    for seq in sequences:
        seq.seq = Seq(str(seq.seq).replace("-", "X"))  # Replace '-' with X'
    return sequences

# Subset sequences
def subset_sequences(sequences, start, end):
    """Extract a subsequence from a list of sequences."""
    subset = []
    for seq in sequences:
        sub_seq = seq.seq[start-1:end]  # Convert to 0-based indexing
        subset.append(SeqRecord(sub_seq, id=seq.id, description=""))
    return subset

In [4]:
def extract_sequences_with_x(sequences):
    """
    Extract sequences that contain the residue 'X'.
    
    Parameters:
    - sequences: list of str, a list of sequences
    
    Returns:
    - list of str: A list of sequences that contain 'X'
    """
    return [seq for seq in sequences if "X" in seq]
def get_complementary_set(test_seq, seq_with_del):
    """
    Get the complementary set of sequences by removing the sequences in seq_with_del from test_seq.
    Always returns the complementary set as SeqRecord objects with meaningful IDs.

    Parameters:
    - test_seq: list of SeqRecord, the original set of sequences
    - seq_with_del: list of SeqRecord, the set of sequences to be excluded

    Returns:
    - list of SeqRecord: The complementary set of sequences with IDs.
    """
    # Create a list of (sequence string, ID) pairs for test_seq
    test_seq_list = [(str(seq.seq), seq.id) for seq in test_seq]
    
    # Create a Counter for test_seq
    test_seq_counts = Counter(test_seq_list)
    
    # Create a Counter for seq_with_del
    seq_with_del_list = [(str(seq.seq), seq.id) for seq in seq_with_del]
    seq_with_del_counts = Counter(seq_with_del_list)
    
    # Subtract the counts
    complementary_counts = test_seq_counts - seq_with_del_counts
    
    # Reconstruct the complementary set as SeqRecord objects
    complementary_sequences = []
    for (seq_str, seq_id), count in complementary_counts.items():
        for _ in range(count):
            complementary_sequences.append(SeqRecord(Seq(seq_str), id=seq_id))
    
    return complementary_sequences

def cluster_sequences_by_position(sequences, position, target_char="X"):
    """
    Cluster sequences based on whether they contain a specific character at a given position.

    Parameters:
    - sequences: list of SeqRecord, the sequences to be clustered.
    - position: int, the 1-based position to check (e.g., 145).
    - target_char: str, the character to match (default is "X").

    Returns:
    - dict: A dictionary with two keys:
        - "contains_target": list of SeqRecord containing the target_char at the position.
        - "does_not_contain_target": list of SeqRecord not containing the target_char at the position.
    """
    contains_target = []

    for seq_record in sequences:
        sequence = str(seq_record.seq)
        # Ensure the position is within the sequence length
        if position - 1 < len(sequence) and sequence[position - 1] == target_char:  # 1-based index to 0-based index
            contains_target.append(seq_record)
    return contains_target

In [8]:
cur_dir = '/Volumes/shared610/Swine_Host/'
out_dir = '/Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8'

In [173]:
fasta_file = cur_dir + 'FASTA_collection/NA/N8_unique.fasta'
ref_file = cur_dir + 'GsGd_residual_scanning/Ref/N8_EPI734757.fasta'

In [175]:
#Extracting binding sites from the entire sequence
start = 1
end = 387
test_seq_full = read_sequences(fasta_file)
test_seq = subset_sequences(test_seq_full, start, end)
ref_seq_full = read_sequences(ref_file)
ref_seq = subset_sequences(ref_seq_full,start, end)

In [178]:
seq_with_del = extract_sequences_with_x(test_seq)
test_seq_with_full = get_complementary_set(test_seq, seq_with_del)

seq_with_del142 = cluster_sequences_by_position(test_seq_full, 142, target_char="X")
seq_with_del143 = cluster_sequences_by_position(test_seq_full, 143, target_char="X")
seq_with_del144 = cluster_sequences_by_position(test_seq_full, 144, target_char="X")

test_seq_with_del142 = subset_sequences(seq_with_del142, start, end)
test_seq_with_del143 = subset_sequences(seq_with_del143, start, end)
test_seq_with_del144 = subset_sequences(seq_with_del144, start, end)

Extracting the mutations compared with reference sequence.

In [181]:
# Numbering adjustment for PDB file
position = 83

In [187]:
# Initialize a dictionary to store mutations for each test sequence
all_mutations = {}
ref_seq_str = str(ref_seq[0].seq)
group = "N8"
for i, seq_record in enumerate(test_seq):
    test_seq_str = str(seq_record.seq)  # Convert current test_seq to a string
    seq_name = seq_record.id 

    # Check if lengths match
    if len(test_seq_str) != len(ref_seq_str):
        print(f"Skipping sequence {i+1}: Length mismatch (test: {len(test_seq_str)}, ref: {len(ref_seq_str)})")
        continue
    
    # Compare sequences and store results
    mutations = compare_sequences(test_seq_str, ref_seq_str, position)
    
    # Parse the accession number (acc) from the sequence name
    try:
        _, _, acc, _ = seq_name.split("|")
    except ValueError:
        print(f"Unable to parse sequence name: {seq_name}")
        acc = f"sequence_{i+1}"  # Fallback name
    
    # Define the output file name using only "acc"
    output_file = os.path.join(out_dir, f"{group}_{acc}.txt")
    
    # Save mutations to the output file
    with open(output_file, "w") as f:
        f.write(mutations)

    print(f"Saved mutations for {acc} to {output_file}.")

Saved mutations for EPI1846961 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1846961.txt.
Saved mutations for EPI1858930 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1858930.txt.
Saved mutations for EPI1916410 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1916410.txt.
Saved mutations for EPI1918889 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1918889.txt.
Saved mutations for EPI1918893 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1918893.txt.
Saved mutations for EPI1918903 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1918903.txt.
Saved mutations for EPI867108 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI867108.txt.
Saved mutations for EPI1185037 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI1185037.txt.
Saved mutations for EPI861011 to /Volumes/shared610/Swine_Host/GsGd_residual_scanning/N8/N8_EPI861011.txt.
Saved mutations for MN7